In [1]:
import sys
print(sys.version)
import numpy
numpy.version.version

3.9.9 (tags/v3.9.9:ccb0e6a, Nov 15 2021, 18:08:50) [MSC v.1929 64 bit (AMD64)]


'1.22.3'

In [2]:
!pip install numpy==1.19.5
!pip install torch torchaudio
!pip install git+https://github.com/huggingface/transformers.git@main
!pip install cython
!pip install madmom
import os
import numpy as np
import torchaudio
import torch
from transformers import AutoProcessor, EncodecModel
import madmom

  Cloning https://github.com/huggingface/transformers.git (to revision main) to c:\users\ripple\appdata\local\temp\pip-req-build-m0_jg_0z
  Resolved https://github.com/huggingface/transformers.git to commit 3ea3ab62d80d91f9bdd16bd3cacd8133fb0d4566
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 2.4/2.4 MB 6.3 MB/s eta 0:00:00


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git 'C:\Users\Ripple\AppData\Local\Temp\pip-req-build-m0_jg_0z'
ERROR: Ignored the following versions that require a different python version: 0.17.0 Requires-Python >=3.8.0; 0.17.0rc0 Requires-Python >=3.8.0; 0.17.1 Requires-Python >=3.8.0; 0.17.2 Requires-Python >=3.8.0; 0.17.3 Requires-Python >=3.8.0; 0.18.0 Requires-Python >=3.8.0; 0.18.0rc0 Requires-Python >=3.8.0; 0.19.0 Requires-Python >=3.8.0; 0.19.0rc0 Requires-Python >=3.8.0; 0.19.1 Requires-Python >=3.8.0; 0.19.2 Requires-Python >=3.8.0; 0.19.3 Requires-Python >=3.8.0; 0.19.4 Requires-Python >=3.8.0; 0.20.0 Requires-Python >=3.8.0; 0.20.0rc0 Requires-Python >=3.8.0; 0.20.0rc1 Requires-Python >=3.8.0; 0.20.1 Requires-Python >=3.8.0; 0.20.2 Requires-Python >=3.8.0; 0.20.3 Requires-Python >=3.8.0; 0.21.0 Requires-Python >=3.8.0; 0.21.0rc0 Requires-Python >=3.8.0; 0.21.1 Requires-Python >=3.8.0; 0.21.2 Requires-Python >=3.8.0; 0.21

# First 10-Second Audio Encoding and Preprocessing for Machine Learning
This script preprocesses an audio dataset for training by encoding the first 10 seconds of each sample using Facebook's EnCodec model to standardize the audio length. It extracts metadata such as BPM, key, and bars from accompanying track_info.txt files and generates one-hot encodings for the music key and track classes. Additionally, the script utilizes Madmom's RNN models to detect downbeats and beats from the audio, leveraging this information to create positional embeddings that represent time-dependent features. The processed data, including audio encodings, positional embeddings, global attributes, and one-hot encodings, is saved for further use, with detailed logs and summaries exported to CSV files. This ensures a structured and consistent dataset for downstream machine learning tasks.

In [ ]:
import os
import numpy as np
import torchaudio
import torch
from transformers import AutoProcessor, EncodecModel, EncodecConfig
import madmom
import csv
import math
import matplotlib.pyplot as plt
import tempfile

def define_keys():
    major_keys = [note + "_MAJOR" for note in ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]]
    minor_keys = [note + "_MINOR" for note in ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]]
    return major_keys + minor_keys

def one_hot_encode_keys(key, all_keys):
    one_hot_vector = np.zeros(len(all_keys))
    if key in all_keys:
        index = all_keys.index(key)
        one_hot_vector[index] = 1
    return one_hot_vector

def get_instrumental_classes(root_dir):
    instrumental_classes = set()
    for root, _, files in os.walk(root_dir):
        if "track_info.txt" in files:
            with open(os.path.join(root, "track_info.txt"), 'r') as f:
                lines = f.readlines()
                for line in lines:
                    if ',' in line:
                        _, attribute = [part.strip() for part in line.split(',', 1)]
                        if 'vocal' not in attribute.lower():
                            instrumental_classes.add(attribute)
    return sorted(instrumental_classes)

def one_hot_encode_track_classes(track_classes, instrumental_classes):
    one_hot_vectors = np.zeros(len(instrumental_classes))
    for track_class in track_classes:
        if track_class in instrumental_classes:
            index = instrumental_classes.index(track_class)
            one_hot_vectors[index] = 1
    return one_hot_vectors

def extract_downbeats(audio_path, fps=100, duration=10):
    # Load the audio file
    audio, rate = torchaudio.load(audio_path)
    
    # Calculate the number of samples for the specified duration
    num_samples = int(duration * rate)
    
    # Slice the audio to the first 10 seconds
    if audio.shape[1] > num_samples:
        audio = audio[:, :num_samples]
    
    # Save the sliced audio to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix='.wav') as temp_audio_file:
        temp_audio_path = temp_audio_file.name
        torchaudio.save(temp_audio_path, audio, rate)
    
    # Process the temporary audio file to extract downbeats
    proc = madmom.features.downbeats.DBNDownBeatTrackingProcessor(beats_per_bar=[4], fps=fps)
    act = madmom.features.downbeats.RNNDownBeatProcessor(fps=fps)(temp_audio_path)
    downbeats = proc(act)
    
    # Filter downbeats to keep only the ones labeled as downbeats
    filtered_downbeats = downbeats[downbeats[:, 1] == 1, 0]
    
    # Delete the temporary file
    os.remove(temp_audio_path)
    
    return filtered_downbeats

def extract_beats(audio_path, fps=100, duration=10):
    # Load the audio file
    audio, rate = torchaudio.load(audio_path)
    
    # Calculate the number of samples for the specified duration
    num_samples = int(duration * rate)
    
    # Slice the audio to the first 10 seconds
    if audio.shape[1] > num_samples:
        audio = audio[:, :num_samples]
    
    # Save the sliced audio to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix='.wav') as temp_audio_file:
        temp_audio_path = temp_audio_file.name
        torchaudio.save(temp_audio_path, audio, rate)
    
    # Process the temporary audio file to extract beats
    beat_act = madmom.features.beats.RNNBeatProcessor(fps=fps)(temp_audio_path)
    beat_proc = madmom.features.beats.BeatDetectionProcessor(fps=fps)
    beats = beat_proc(beat_act)
    
    # Delete the temporary file
    os.remove(temp_audio_path)
    
    return beats


def create_positional_embeddings(beat_times, downbeat_times, audio_duration, fps=75, K=32):
    total_frames = int(np.ceil(audio_duration * fps))
    print ('total frames: ', total_frames)
    
    def ramps(positions, size):
        result = np.zeros(size)
        for a, b in zip(positions[:-1], positions[1:]):
            result[a:b] = np.linspace(0, 1, b - a, endpoint=False)
        missing = positions[0]
        if missing:
            piece = result[positions[0]:positions[1]]
            pieces = np.tile(piece, missing // len(piece) + 1)
            result[:missing] = pieces[-missing:]
        missing = size - positions[-1]
        if missing:
            piece = result[positions[-2]:positions[-1]]
            pieces = np.tile(piece, missing // len(piece) + 1)
            result[-missing:] = pieces[:missing]
        return result

    time_vector = np.linspace(0, audio_duration, total_frames)
    vector_downbeat = ramps((downbeat_times * fps).astype(int), total_frames)
    vector_beat = ramps((beat_times * fps).astype(int), total_frames)

    frequencies = np.arange(1, K + 1)
    embeddings_downbeat = []
    embeddings_beat = []

    for k in frequencies:
        embeddings_downbeat.append(np.sin(2 * np.pi * vector_downbeat * k))
        embeddings_downbeat.append(np.cos(2 * np.pi * vector_downbeat * k))
        embeddings_beat.append(np.sin(2 * np.pi * vector_beat * k))
        embeddings_beat.append(np.cos(2 * np.pi * vector_beat * k))

    embeddings_downbeat = np.stack(embeddings_downbeat, axis=1)
    embeddings_beat = np.stack(embeddings_beat, axis=1)
    embeddings = np.hstack((embeddings_downbeat, embeddings_beat))

    return torch.from_numpy(embeddings).float()

def get_global_attributes(track_info_path):
    global_attributes = {}
    with open(track_info_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if ':' in line:
                key, value = line.strip().split(':', 1)
                global_attributes[key.strip().lower()] = value.strip()
    return global_attributes

def encode_audio(audio_path):
    audio, rate = torchaudio.load(audio_path)
    
    # Calculate the number of samples corresponding to 10 seconds
    max_length_in_samples = int(rate * 10)
    
    # Trim or pad the audio to ensure it's exactly 10 seconds long
    if audio.shape[1] > max_length_in_samples:
        audio = audio[:, :max_length_in_samples]
    else:
        pad_length = max_length_in_samples - audio.shape[1]
        audio = torch.nn.functional.pad(audio, (0, pad_length))
    
    # Ensure that the audio has a single channel
    if audio.ndim > 1:
        audio = audio[0]
    
    audio_np = audio.numpy()
    inputs = processor(audio_np, sampling_rate=rate, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.encode(inputs["input_values"], inputs["padding_mask"], 3)
    length_in_seconds = min(audio.shape[0] / rate, 10)  # Ensure length is capped at 10 seconds
    return outputs.audio_codes.squeeze(), length_in_seconds

def process_folder(folder_path, instrumental_classes, fps=100):
    folder_data = {
        'generation_data': {},
        'positional_embedding': None,
        'global': {},
        'track': None,
        'key_one_hot': None
    }
    
    track_info_path = os.path.join(folder_path, "track_info.txt")
    all_keys = define_keys()
    folder_data['global'] = get_global_attributes(track_info_path)
    bars, tempo = int(folder_data['global'].get('bars', 0)), float(folder_data['global'].get('bpm', 120))
    music_key = folder_data['global'].get('key', "").replace(" ", "_").upper()
    
    if music_key:
        folder_data['key_one_hot'] = one_hot_encode_keys(music_key, all_keys)
        print('key_one_hot', folder_data['key_one_hot'].shape)
    
    filename_to_track_class = {}
    with open(track_info_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if ',' in line:
                file_name, track_class = [part.strip() for part in line.split(',', 1)]
                filename_to_track_class[file_name] = track_class
                
    for file_name, track_class in filename_to_track_class.items():
        audio_path = os.path.join(folder_path, file_name)
        if os.path.exists(audio_path) and file_name.endswith(".wav"):
            audio_code, length_in_seconds = encode_audio(audio_path)
            folder_data['generation_data'][track_class] = audio_code
            if track_class == 'full_instrumental':
                downbeats = extract_downbeats(audio_path, fps)
                beats = extract_beats(audio_path, fps)
                folder_data['positional_embedding'] = create_positional_embeddings(beats, downbeats, length_in_seconds)
                print('pos, audio', folder_data['positional_embedding'].shape, audio_code.shape, length_in_seconds)
                
    track_classes = [track_class for file_name, track_class in filename_to_track_class.items() if 'vocal' not in track_class.lower()]
    folder_data['track'] = one_hot_encode_track_classes(track_classes, instrumental_classes)
    
    return folder_data

def process_tracks(root_dir, fps=100):
    instrumental_classes = get_instrumental_classes(root_dir)
    all_folders_data = {}
    
    for root, dirs, files in os.walk(root_dir):
        if "track_info.txt" in files:
            folder_data = process_folder(root, instrumental_classes, fps)
            all_folders_data[os.path.basename(root)] = folder_data
            
    return all_folders_data

def save_track_classes_to_csv(instrumental_classes, output_file='track_classes.csv'):
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Track Class', 'One Hot Encoding'])
        for index, track_class in enumerate(instrumental_classes):
            one_hot_vector = [0] * len(instrumental_classes)
            one_hot_vector[index] = 1  # Set the current class index to 1
            writer.writerow([track_class, one_hot_vector])

def save_global_attributes_to_csv(all_folders_data, output_file='global_attributes.csv'):
    with open(output_file, mode='w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Folder Name', 'BPM', 'Bars', 'Key', 'Music Key One Hot', 'Track Class One Hot'])
        for folder_name, data in all_folders_data.items():
            global_attributes = data['global']
            bpm = global_attributes.get('bpm', 'N/A')
            bars = global_attributes.get('bars', 'N/A')
            key = global_attributes.get('key', 'N/A').replace('_', ' ').title()
            key_one_hot = data['key_one_hot']
            track_class_one_hot = data['track']
            key_one_hot_str = ', '.join(map(str, key_one_hot))
            track_class_one_hot_str = ', '.join(map(str, track_class_one_hot))
            writer.writerow([folder_name, bpm, bars, key, key_one_hot_str, track_class_one_hot_str])

def log_sample_features(folder_data, log_file='sample_features_log.csv'):
    with open(log_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([
            'Folder', 'File Name', 'Track Class', 'Audio Code Shape', 'Audio Scales Shape',
            'Positional Embedding Shape', 'Global Attributes', 'Key One Hot Encoding Shape', 'Track Class One Hot Encoding Shape'
        ])

        for folder, data in folder_data.items():
            global_attrs = ', '.join([f"{key}: {value}" for key, value in data['global'].items()])
            key_one_hot_shape = data['key_one_hot'].shape if data['key_one_hot'] is not None else 'N/A'
            track_class_one_hot_shape = data['track'].shape if data['track'] is not None else 'N/A'
            for file_name, details in data['generation_data'].items():
                audio_codes_shape = details[0].shape if details[0] is not None else 'N/A'
                audio_scales_shape = len(details[1]) if details[1] is not None else 'N/A'
                pos_emb_shape = data['positional_embedding'].shape if data['positional_embedding'] is not None else 'N/A'
                writer.writerow([
                    folder, file_name, file_name.split('.')[0], audio_codes_shape, audio_scales_shape,
                    pos_emb_shape, global_attrs, key_one_hot_shape, track_class_one_hot_shape
                ])
# Initialize Encodec Model and Processor
processor = AutoProcessor.from_pretrained("facebook/encodec_24khz")
        
model = EncodecModel.from_pretrained("facebook/encodec_24khz")


# Example usage
root_dir = '../processed_dataset/'  # Adjust this path to your dataset location
all_data = process_tracks(root_dir)
print(all_data, len(all_data))
log_sample_features(all_data)

# Save track classes and global attributes to CSV
instrumental_classes = get_instrumental_classes(root_dir)
save_track_classes_to_csv(instrumental_classes)
save_global_attributes_to_csv(all_data)

In [41]:
import json
import numpy as np

def convert_data_for_npy(data):
    """Recursively convert data types in the nested dictionary for numpy compatibility."""
    if isinstance(data, dict):
        return {k: convert_data_for_npy(v) for k, v in data.items()}
    elif isinstance(data, (list, np.ndarray)):
        return np.array([convert_data_for_npy(item) for item in data])
    elif isinstance(data, torch.Tensor):
        return data.detach().cpu().numpy()
    else:
        return data

converted_data = convert_data_for_npy(all_data)

output_file_name_npy = 'processed_tracks_data_final_10secs_embeddings_standardized.npy'
np.save(output_file_name_npy, np.array(converted_data, dtype=object))

print(f"Data saved to {output_file_name_npy}")

Data saved to processed_tracks_data_final_10secs_embeddings_standardized.npy


# Enhanced Audio Preprocessing with 10-Second Chunks for Richer Dataset
This script preprocesses an audio dataset for training by dividing each audio track into 10-second chunks, significantly increasing the number of data samples compared to the earlier script, which processed only the first 10 seconds of each track. It encodes each chunk using Facebook's EnCodec model, ensuring that all audio segments are standardized in length. The script extracts metadata such as BPM, key, and bars from accompanying track_info.txt files and generates one-hot encodings for the music key and track classes.

For tracks labeled as full_instrumental, the script calculates downbeats and beats using Madmom's RNN models, leveraging this information to create detailed positional embeddings for each chunk. By processing each track into multiple 10-second chunks, the script provides a richer dataset with finer granularity, enhancing its utility for machine learning models. Each chunk's data, including audio encodings, positional embeddings, and associated metadata, is aggregated and saved for efficient downstream usage, enabling better model training outcomes. This approach contrasts with the first script by maximizing data diversity and volume while maintaining temporal and structural consistency.

In [27]:
import os
import numpy as np
import torchaudio
import torch
from transformers import AutoProcessor, EncodecModel
import madmom
import tempfile

# Define helper functions
def get_global_attributes(track_info_path):
    global_attributes = {}
    with open(track_info_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if ':' in line:
                key, value = line.strip().split(':', 1)
                global_attributes[key.strip().lower()] = value.strip()
    return global_attributes

def define_keys():
    major_keys = [note + "_MAJOR" for note in ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]]
    minor_keys = [note + "_MINOR" for note in ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]]
    return major_keys + minor_keys

def one_hot_encode_keys(key, all_keys):
    one_hot_vector = np.zeros(len(all_keys))
    if key in all_keys:
        index = all_keys.index(key)
        one_hot_vector[index] = 1
    return one_hot_vector

def get_instrumental_classes(root_dir):
    instrumental_classes = set()
    for root, _, files in os.walk(root_dir):
        if "track_info.txt" in files:
            with open(os.path.join(root, "track_info.txt"), 'r') as f:
                lines = f.readlines()
                for line in lines:
                    if ',' in line:
                        _, attribute = [part.strip() for part in line.split(',', 1)]
                        if 'vocal' not in attribute.lower():
                            instrumental_classes.add(attribute)
    return sorted(instrumental_classes)

def one_hot_encode_track_classes(track_classes, instrumental_classes):
    one_hot_vectors = np.zeros(len(instrumental_classes))
    for track_class in track_classes:
        if track_class in instrumental_classes:
            index = instrumental_classes.index(track_class)
            one_hot_vectors[index] = 1
    return one_hot_vectors

def split_audio_into_chunks(audio, rate, chunk_duration=10):
    chunk_length = int(chunk_duration * rate)
    num_chunks = audio.shape[1] // chunk_length
    chunks = [audio[:, i * chunk_length:(i + 1) * chunk_length] for i in range(num_chunks)]
    return chunks

def extract_downbeats(audio_path, fps=100):
    proc = madmom.features.downbeats.DBNDownBeatTrackingProcessor(beats_per_bar=[4], fps=fps)
    act = madmom.features.downbeats.RNNDownBeatProcessor(fps=fps)(audio_path)
    downbeats = proc(act)
    return downbeats[downbeats[:, 1] == 1, 0]

def extract_beats(audio_path, fps=100):
    beat_act = madmom.features.beats.RNNBeatProcessor(fps=fps)(audio_path)
    beat_proc = madmom.features.beats.BeatDetectionProcessor(fps=fps)
    return beat_proc(beat_act)

def create_positional_embeddings(beat_times, downbeat_times, audio_duration, fps=75, K=32):
    total_frames = int(np.ceil(audio_duration * fps))
    
    def ramps(positions, size):
        result = np.zeros(size)
        for a, b in zip(positions[:-1], positions[1:]):
            result[a:b] = np.linspace(0, 1, b - a, endpoint=False)
        return result

    time_vector = np.linspace(0, audio_duration, total_frames)
    vector_downbeat = ramps((downbeat_times * fps).astype(int), total_frames)
    vector_beat = ramps((beat_times * fps).astype(int), total_frames)

    frequencies = np.arange(1, K + 1)
    embeddings_downbeat = []
    embeddings_beat = []

    for k in frequencies:
        embeddings_downbeat.append(np.sin(2 * np.pi * vector_downbeat * k))
        embeddings_downbeat.append(np.cos(2 * np.pi * vector_downbeat * k))
        embeddings_beat.append(np.sin(2 * np.pi * vector_beat * k))
        embeddings_beat.append(np.cos(2 * np.pi * vector_beat * k))

    embeddings_downbeat = np.stack(embeddings_downbeat, axis=1)
    embeddings_beat = np.stack(embeddings_beat, axis=1)
    embeddings = np.hstack((embeddings_downbeat, embeddings_beat))
    
    num_zeros = np.sum(embeddings == 0)
    print(f"[DEBUG] Final positional embeddings shape: {embeddings.shape}")
    print(f"[DEBUG] Number of zero elements in final positional embeddings: {num_zeros}")

    return torch.from_numpy(embeddings).float()

def encode_audio_chunk(audio_chunk, rate):
    # Ensure the audio has a single channel
    if audio_chunk.ndim > 1:
        audio_chunk = audio_chunk[0]
    
    audio_np = audio_chunk.numpy()
    inputs = processor(audio_np, sampling_rate=rate, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model.encode(inputs["input_values"], inputs["padding_mask"], 3)
    return outputs.audio_codes.squeeze()

def process_audio_chunks(audio_path, track_class, is_full_instrumental, global_attributes, key_one_hot, track_one_hot, fps_beats=100, fps_positional=75, chunk_duration=10):
    """
    Processes audio chunks for a specific track_class and ensures all track_classes' audio codes
    are accumulated in generation_data.
    """
    audio, rate = torchaudio.load(audio_path)
    chunks = split_audio_into_chunks(audio, rate, chunk_duration)
    samples = []
    
    for i, chunk in enumerate(chunks):
        audio_code = encode_audio_chunk(chunk, rate)
        if audio_code.shape[1] == int(chunk_duration * fps_positional):  # Valid 10-second sample
            sample = {
                'generation_data': {},  # Initialize empty dict for generation_data
                'chunk_index': i,
                'bars': global_attributes.get('bars', 0),
                'key_one_hot': key_one_hot,
                'track_one_hot': track_one_hot
            }
            
            # Add the current track_class audio code to generation_data
            sample['generation_data'][track_class] = np.array(audio_code)
            
            if is_full_instrumental:  # Compute positional embeddings only for full_instrumental
                with tempfile.NamedTemporaryFile(delete=False, suffix='.wav') as temp_audio_file:
                    temp_audio_path = temp_audio_file.name
                    torchaudio.save(temp_audio_path, chunk, rate)
                
                # Compute beats and downbeats
                downbeats = extract_downbeats(temp_audio_path, fps=fps_beats)
                beats = extract_beats(temp_audio_path, fps=fps_beats)
                os.remove(temp_audio_path)
                
                # Compute positional embedding for the chunk
                positional_embedding = create_positional_embeddings(beats, downbeats, chunk_duration, fps=fps_positional)
                sample['positional_embedding'] = np.array(positional_embedding)
            
            samples.append(sample)
    
    return samples



def process_folder_with_chunks(folder_path, instrumental_classes, fps_beats=100, fps_positional=75, chunk_duration=10):
    """
    Processes a folder and outputs individual chunks as separate samples with generation_data.
    Ensures that all track_classes' audio codes are accumulated in generation_data.
    """
    all_samples = []
    track_info_path = os.path.join(folder_path, "track_info.txt")
    global_attributes = get_global_attributes(track_info_path)
    all_keys = define_keys()
    
    music_key = global_attributes.get('key', "").replace(" ", "_").upper()
    key_one_hot = one_hot_encode_keys(music_key, all_keys)
    
    filename_to_track_class = {}
    with open(track_info_path, 'r') as f:
        for line in f.readlines():
            if ',' in line:
                file_name, track_class = [part.strip() for part in line.split(',', 1)]
                filename_to_track_class[file_name] = track_class

    # Process 'full_instrumental' track first to ensure positional_embedding is included
    track_classes_ordered = sorted(filename_to_track_class.items(), key=lambda x: 0 if x[1].lower() == 'full_instrumental' else 1)
    
    # Initialize an empty dictionary to accumulate data for each chunk index
    chunk_samples = {}
    
    for file_name, track_class in track_classes_ordered:
        audio_path = os.path.join(folder_path, file_name)
        if os.path.exists(audio_path) and file_name.endswith(".wav"):
            is_full_instrumental = track_class.lower() == 'full_instrumental'
            track_one_hot = one_hot_encode_track_classes([track_class], instrumental_classes)
            
            # Process chunks for the current track_class
            samples = process_audio_chunks(
                audio_path,
                track_class,
                is_full_instrumental,
                global_attributes,
                key_one_hot,
                track_one_hot,
                fps_beats,
                fps_positional,
                chunk_duration
            )
            
            # Aggregate data across track_classes for the same chunk index
            for sample in samples:
                chunk_index = sample['chunk_index']
                if chunk_index not in chunk_samples:
                    chunk_samples[chunk_index] = sample
                else:
                    # Merge generation_data from the new sample into existing one
                    chunk_samples[chunk_index]['generation_data'].update(sample['generation_data'])
                    # Update positional_embedding if present
                    if 'positional_embedding' in sample:
                        chunk_samples[chunk_index]['positional_embedding'] = sample['positional_embedding']
    
    # Convert aggregated samples back to a list
    all_samples = list(chunk_samples.values())
    return all_samples




def process_tracks_with_chunks(root_dir, fps_beats=100, fps_positional=75, chunk_duration=10):
    instrumental_classes = get_instrumental_classes(root_dir)
    all_samples = {}
    sample_id = 0
    
    for root, _, files in os.walk(root_dir):
        if "track_info.txt" in files:
            folder_samples = process_folder_with_chunks(root, instrumental_classes, fps_beats, fps_positional, chunk_duration)
            
            # Assign unique IDs to each sample
            for sample in folder_samples:
                all_samples[f"sample_{sample_id}"] = sample
                sample_id += 1
    
    return all_samples

# Initialize Encodec Processor and Model
processor = AutoProcessor.from_pretrained("facebook/encodec_24khz")
model = EncodecModel.from_pretrained("facebook/encodec_24khz")

# Example Usage
root_dir = '../processed_dataset'  # Update path as needed
all_data = process_tracks_with_chunks(root_dir)

all_data

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


[DEBUG] Final positional embeddings shape: (750, 128)
[DEBUG] Number of zero elements in final positional embeddings: 3232
[DEBUG] Final positional embeddings shape: (750, 128)
[DEBUG] Number of zero elements in final positional embeddings: 7744


KeyboardInterrupt: 

In [40]:
import numpy as np
import torch

def convert_to_npy(data):
    """
    Recursively converts the output data structure into a numpy-compatible format
    for saving to .npy files.
    """
    if isinstance(data, dict):
        # Recursively process dictionary values
        return {key: convert_to_npy(value) for key, value in data.items()}
    elif isinstance(data, (list, tuple)):
        # Convert lists/tuples to numpy arrays
        return np.array([convert_to_npy(item) for item in data], dtype=object)
    elif isinstance(data, torch.Tensor):
        # Convert torch.Tensor to numpy array
        return data.detach().cpu().numpy()
    elif isinstance(data, np.ndarray):
        # If already a numpy array, return as is
        return data
    else:
        # Scalars and other types (e.g., int, float, str)
        return data

    
# Convert the processed data into a numpy-compatible format
converted_data = convert_to_npy(all_data)

# Save the converted data to a .npy file
output_file_name = 'fulldataset_10sec_positional_embs.npy'
np.save(output_file_name, converted_data)

print(f"Data saved to {output_file_name}")


Data saved to fulldataset_10sec_positional_embs.npy
